# Define Function To Pull Tweets From A Specified User Using Tweepy

In [ ]:
import tweepy
import re

# Twitter API credentials
consumer_key = "1kmJBTjd214v2eVIi7tUDpzdr"
consumer_secret = "M4Pqx3Yvxid1yIWFJBAPwdYxJpP3XlBwtd3bfywndxI0QswpGk"
access_key = "276159644-CGZeVcjXsbNetX4JFlXe3ML4wZitAOdunbHGOazM"
access_secret = "yPltYhYf7nIH3TGbNwrTvxtCzOhNukuw9gIf2LfZEjbMy"


def get_all_tweets(screen_name):
    
    # Twitter only allows access to a users most recent 3240 tweets with this method

    # Authorize Twitter, initialize Tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)

    # Create a list to hold all the tweets
    alltweets = []

    # Make initial request for most recent tweets
    new_tweets = api.user_timeline(screen_name = screen_name, count = 200, include_rts = False)

    # Store tweets in list
    alltweets.extend(new_tweets)

    # Save the id of the oldest tweet minus one
    oldest = alltweets[-1].id - 1

    # Keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print ("getting tweets before %s" % (oldest))

        # All subsequent requests use the max_id parameter to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name, count = 200, max_id = oldest)

        # Save most recent tweets
        alltweets.extend(new_tweets)

        # Update the ID of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print ("...%s tweets downloaded so far"  % (len(alltweets)))

    # Create list to hold the 'text' portion of the tweet's JSON data
    outtweets = []
    for tweet in alltweets :
       outtweets.append(tweet.text)

    # Test the output for 'outtweets' 
    print()
    print(outtweets[:10])
    print()
    print(len(outtweets))
    
    # Write tweets to text file, with one word on each line
    output_file_name = screen_name + "_tweets.txt"
    delimiter = "\n"

    with open(output_file_name, 'w') as f:
            f.write(delimiter.join(word for word in outtweets))
    

    
                

    


if __name__ == '__main__':
    #pass in the username of the account you want to download
    get_all_tweets("UncleChaps")

# Take Words From Text File, Append To List, and Modify Strings

In [ ]:
wd = "/Users/MikeRalston/Desktop/ADA/API : Text Mining Final Project/"

# Create list to store words from text file
tweet_words = []

# Open specified user's tweet file, split each line up, and append 'tweet_words' with contents of 'line'
with open(wd + "UncleChaps_tweets.txt", 'r') as input_file:
    for idx, line in enumerate(input_file.readlines()):
        line = line.strip()
        tweet_words.append(line)
        
# Split line up into individual words, cast to lowercase, remove empty strings 
tweet_words = [word for word in tweet_words for word in word.split()]
tweet_words = [word.lower() for word in tweet_words] 
tweet_words = [word for word in tweet_words if word]

# Remove links, remove mentions, remove hashtags
tweet_words = [word for word in tweet_words if "http" not in word ]
tweet_words = [word for word in tweet_words if "@" not in word ]
tweet_words = [word for word in tweet_words if "#" not in word ]

# Remove punctuation
tweet_words = [word.replace("'", "") for word in tweet_words]
tweet_words = [word.replace(".", "") for word in tweet_words]
tweet_words = [word.replace(",", "") for word in tweet_words]
tweet_words = [word.replace(":", "") for word in tweet_words]
tweet_words = [word.replace("-", "") for word in tweet_words]
tweet_words = [word.replace("'", "") for word in tweet_words]

# Test output of 'tweet_words'
print(tweet_words[:100])


# Create Dictionary To Hold Word-Scores From Tidytext Sentiment Document

In [ ]:
# Create dictionary to hold sentiment scores
sentiment_scores = {}

# If word from tweet_words appears in the sentiments file, the appropriaye score is assigned
with open(wd + "tidytext_sentiments.txt",'r') as infile :
    next(infile)
    for line in infile.readlines() :
        line = line.strip().split("\t")
        if line[1] == "positive" :
            sentiment_scores[line[0]] = 1
        else :
            sentiment_scores[line[0]] = -1


# Test scoring system for words
for idx, word in enumerate(sentiment_scores) :
    print("{} has score {}.".format(word,sentiment_scores[word]))
    if idx > 10 :
        break

# Score Tweet_Words[ ]

In [ ]:
# Create list with the same length as the tweet_words list
scores = [0] * len(tweet_words)

# Set current score to zero
current_score = 0 

# Iterate over each word in tweet_words and keep and running score of the words
for idx, word in enumerate(tweet_words) :
    if word in sentiment_scores :
        current_score += sentiment_scores[word.lower()]
   
    scores[idx] = current_score

# Write results out to a text file for input into R/GGPlot
with open(wd + "UncleChaps_tweet_scores.txt",'w') as ofile :
    ofile.write("word\tscore\n")
    for idx, score in enumerate(scores) :
        ofile.write("\t".join([str(idx+1),str(score)]) + "\n")  